In [2]:
!pip install matplotlib

  Using cached matplotlib-3.10.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (106 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pillow-11.2.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
Using cached contourpy-1.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (323 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.58.1-cp312-cp312-manylinux_2_5_x86_64

In [10]:
# Célula 1: Configuração e Definição dos Dados Paralelos

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt # Para visualização, se necessário

# Dados de exemplo: Pares de frases em inglês (entrada) e português (saída).
# Para tradução, precisamos de dados paralelos.
# Note o uso de tokens especiais [START] e [END] para a sequência de destino.
# Estes tokens ajudam o decodificador a saber quando iniciar e parar a geração.
raw_data = [
    ["I am a student.", "Eu sou um estudante."],
    ["You are a teacher.", "Você é um professor."],
    ["He is a doctor.", "Ele é um médico."],
    ["She is happy.", "Ela está feliz."],
    ["We are friends.", "Nós somos amigos."],
    ["They are playing.", "Eles estão jogando."],
    ["The cat is black.", "O gato é preto."],
    ["The dog is big.", "O cachorro é grande."],
    ["I like programming.", "Eu gosto de programar."],
    ["She loves music.", "Ela ama música."],
    ["He reads books.", "Ele lê livros."],
    ["We eat food.", "Nós comemos comida."]
]

# Adicionando tokens de início e fim às frases de destino
# Isso é crucial para o treinamento e inferência do decodificador.
input_texts = [pair[0] for pair in raw_data]
target_texts = ["[START] " + pair[1] + " [END]" for pair in raw_data]

print(f"Número de pares de frases: {len(raw_data)}")
print(f"Exemplo de entrada: '{input_texts[0]}'")
print(f"Exemplo de saída (com tokens): '{target_texts[0]}'")


Número de pares de frases: 12
Exemplo de entrada: 'I am a student.'
Exemplo de saída (com tokens): '[START] Eu sou um estudante. [END]'


In [11]:
# Célula 2: Pré-processamento dos Dados

# 1. Tokenização para a Língua de Entrada (Inglês)
# Cria um tokenizador para as frases de entrada.
# Modificado: Adicionado filters='' para não remover caracteres especiais como []
input_tokenizer = Tokenizer(filters='')
input_tokenizer.fit_on_texts(input_texts)
input_word_index = input_tokenizer.word_index
num_encoder_tokens = len(input_word_index) + 1 # +1 para o índice 0 (padding)

# Converte as frases de entrada em sequências de inteiros.
encoder_input_sequences = input_tokenizer.texts_to_sequences(input_texts)

# 2. Tokenização para a Língua de Saída (Português)
# Cria um tokenizador para as frases de destino.
# Modificado: Adicionado filters='' para não remover caracteres especiais como []
target_tokenizer = Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_texts)
target_word_index = target_tokenizer.word_index
num_decoder_tokens = len(target_word_index) + 1 # +1 para o índice 0 (padding)

# Converte as frases de destino em sequências de inteiros.
decoder_input_sequences = target_tokenizer.texts_to_sequences(target_texts)

# 3. Determinar o Comprimento Máximo das Sequências
# O comprimento máximo das sequências de entrada e saída é usado para o padding.
max_encoder_seq_length = max([len(seq) for seq in encoder_input_sequences])
max_decoder_seq_length = max([len(seq) for seq in decoder_input_sequences])

print(f"Número de tokens do encoder (Inglês): {num_encoder_tokens}")
print(f"Número de tokens do decoder (Português): {num_decoder_tokens}")
print(f"Comprimento máximo da sequência de entrada: {max_encoder_seq_length}")
print(f"Comprimento máximo da sequência de saída: {max_decoder_seq_length}")

# 4. Padding das Sequências
# Pad_sequences adiciona zeros no início das sequências para que todas tenham o mesmo comprimento.
encoder_input_data = pad_sequences(encoder_input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_sequences, maxlen=max_decoder_seq_length, padding='post')

# 5. Preparar Dados de Saída do Decodificador (Target Data)
# O target data do decodificador é a mesma sequência de entrada do decodificador,
# mas SHIFTADA em uma posição para a frente. Isso é o "Teacher Forcing".
# Ex: Entrada: "[START] Eu sou um estudante [END]"
#     Saída:   "Eu sou um estudante [END] [PAD]"
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32'
)

# Converte o decoder_target_data para one-hot encoding
for i, seq in enumerate(decoder_input_sequences):
    for t, word_id in enumerate(seq):
        if t > 0: # Ignora o token [START] na saída
            decoder_target_data[i, t - 1, word_id] = 1.0

print(f"\nFormato dos dados de entrada do encoder: {encoder_input_data.shape}")
print(f"Formato dos dados de entrada do decoder: {decoder_input_data.shape}")
print(f"Formato dos dados de saída do decoder (one-hot): {decoder_target_data.shape}")


Número de tokens do encoder (Inglês): 30
Número de tokens do decoder (Português): 35
Comprimento máximo da sequência de entrada: 4
Comprimento máximo da sequência de saída: 6

Formato dos dados de entrada do encoder: (12, 4)
Formato dos dados de entrada do decoder: (12, 6)
Formato dos dados de saída do decoder (one-hot): (12, 6, 35)


In [12]:
# Célula 3: Arquitetura do Modelo Seq2Seq (Encoder-Decoder)

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# Dimensão do espaço latente (tamanho do vetor de contexto da LSTM)
latent_dim = 256 # Um tamanho comum para a dimensão dos estados internos da LSTM.

# --- ENCODER ---
# Input do Encoder: Sequências de inteiros da língua de origem.
encoder_inputs = Input(shape=(None,)) # 'None' permite sequências de comprimento variável.
# Camada de Embedding do Encoder: Converte IDs de palavras em vetores densos.
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
# Camada LSTM do Encoder: Processa a sequência de entrada.
# return_state=True: Retorna os estados internos (h e c) da LSTM, que serão passados para o decodificador.
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
# states: O vetor de contexto do encoder, contendo a "memória" da sequência de entrada.
encoder_states = [state_h, state_c]

# --- DECODER ---
# Input do Decoder: Sequências de inteiros da língua de destino (com [START]/[END] tokens).
decoder_inputs = Input(shape=(None,))
# Camada de Embedding do Decoder: Converte IDs de palavras em vetores densos para o decodificador.
decoder_embedding = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
# Camada LSTM do Decoder: Gera a sequência de saída.
# return_sequences=True: Retorna a sequência completa de saídas para cada passo de tempo.
# return_state=True: Retorna os estados internos (h e c) para o loop de inferência.
# initial_state=encoder_states: O decodificador começa com a "memória" do codificador.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
# Camada Densa de Saída: Classifica a próxima palavra no vocabulário de destino.
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# --- MODELO COMPLETO DE TREINAMENTO ---
# Este modelo aceita as sequências de entrada do encoder e do decoder,
# e tenta prever a próxima palavra da sequência de destino.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilação do Modelo
# loss='categorical_crossentropy': Usada para saídas one-hot encoded.
# optimizer='rmsprop': Um otimizador comum para RNNs, mas 'adam' também funciona bem.
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Exibe um resumo da arquitetura do modelo
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 256) │      7,680 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 256) │      8,960 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    525,312 │ embedding_2[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    525,312 │ embedding_3[0][0… │
│                     │ 256), (None,      │            │ lstm_2[0][1],     │
│                     │ 256), (None,      │            │ lstm_2[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 35)  │      8,995 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,076,259 (4.11 MB)

 Trainable params: 1,076,259 (4.11 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Célula 4: Treinamento do Modelo

print("Iniciando o treinamento do modelo Seq2Seq...")

# Parâmetros de treinamento
batch_size = 64 # Número de amostras processadas por vez.
epochs = 100    # Número de vezes que o modelo verá todo o dataset.

# model.fit: Treina o modelo.
# inputs: [dados de entrada do encoder, dados de entrada do decoder]
# target: dados de saída do decoder (one-hot encoded)
# verbose=1: Exibe o progresso do treinamento em cada época.
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2 # Usa 20% dos dados para validação.
)

print("\nTreinamento concluído.")

# Opcional: Visualizar o histórico de treinamento
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label='val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()


Iniciando o treinamento do modelo Seq2Seq...
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0185 - loss: 2.7652 - val_accuracy: 0.1667 - val_loss: 2.3656
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1667 - loss: 2.7470 - val_accuracy: 0.1667 - val_loss: 2.3604
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.1667 - loss: 2.7315 - val_accuracy: 0.1667 - val_loss: 2.3551
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.1667 - loss: 2.7157 - val_accuracy: 0.1667 - val_loss: 2.3492
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.1667 - loss: 2.6982 - val_accuracy: 0.1667 - val_loss: 2.3420
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.1667 - loss: 2.6776 - val_accuracy: 0.1667 - val_loss: 2.3328
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.1667 - loss: 2.6519 - val_accuracy: 0.1667 - val_loss: 2.3203
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1667 - los

In [15]:
# Célula 5: Configuração do Modelo para Inferência (Predição)

# Para a inferência (tradução de novas frases), precisamos de modelos separados
# que nos permitam alimentar o decodificador passo a passo.

# --- Modelo do Encoder (para inferência) ---
# Aceita a sequência de entrada e retorna apenas os estados finais do encoder.
encoder_model = Model(encoder_inputs, encoder_states)

# --- Modelo do Decoder (para inferência) ---
# O decodificador agora aceitará:
# 1. A entrada de um único passo de tempo (a palavra gerada anteriormente ou [START]).
# 2. Os estados internos (h e c) do passo anterior (ou os estados finais do encoder).
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h_decoder, state_c_decoder = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h_decoder, state_c_decoder]
decoder_outputs = decoder_dense(decoder_outputs)

# Este modelo do decodificador retorna:
# 1. As probabilidades da próxima palavra.
# 2. Seus estados internos atualizados (para o próximo passo de tempo).
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

print("Modelos de Encoder e Decoder para inferência configurados.")


Modelos de Encoder e Decoder para inferência configurados.


In [16]:
# Célula 6: Função de Tradução (Inferência)

# Mapeamento de ID para Palavra para o vocabulário de destino (Português)
reverse_target_word_index = dict(
    (i, word) for word, i in target_word_index.items()
)

def decode_sequence(input_seq):
    """
    Traduz uma sequência de entrada (Inglês) para uma sequência de saída (Português).

    Args:
        input_seq: Sequência de inteiros da frase de entrada (Inglês).

    Returns:
        A frase traduzida em Português como string.
    """
    # 1. Codifica a sequência de entrada para obter os estados iniciais do decodificador.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # 2. Prepara a sequência de entrada do decodificador com o token [START].
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word_index['[start]'] # Inicia com o token [START]

    # 3. Loop de amostragem para gerar a sequência de saída.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # Preveja a próxima palavra e os estados atualizados do decodificador.
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Amostra a próxima palavra (pega a palavra com maior probabilidade).
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Condição de parada: Se a palavra for [END] ou o comprimento máximo for atingido.
        if sampled_word == '[end]' or len(decoded_sentence.split()) > max_decoder_seq_length:
            stop_condition = True

        # Atualiza a sequência de entrada do decodificador para o próximo passo.
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Atualiza os estados para o próximo passo.
        states_value = [h, c]
    
    # Remove o token [END] da frase final.
    return decoded_sentence.replace(' [end]', '').strip()

print("Função 'decode_sequence' definida.")


Função 'decode_sequence' definida.


In [17]:
# Célula 7: Teste e Demonstração da Tradução

print("\n--- Testando o Modelo de Tradução ---")

# Testar com algumas frases do dataset de treinamento
for seq_index in range(len(input_texts)):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    translated_sentence = decode_sequence(input_seq)
    
    print(f"Original (Inglês): '{input_texts[seq_index]}'")
    print(f"Esperado (Português): '{target_texts[seq_index].replace('[START] ', '').replace(' [END]', '')}'")
    print(f"Traduzido (Português): '{translated_sentence}'")
    print("-" * 30)

# Testar com uma frase nova (que o modelo não viu durante o treinamento)
# A capacidade de generalização será limitada com um dataset tão pequeno.
new_english_sentence = "I am happy."
# Primeiro, tokenizar e pad a nova frase.
new_input_seq = input_tokenizer.texts_to_sequences([new_english_sentence])
new_input_data = pad_sequences(new_input_seq, maxlen=max_encoder_seq_length, padding='post')

translated_new_sentence = decode_sequence(new_input_data)
print(f"\nNova frase (Inglês): '{new_english_sentence}'")
print(f"Traduzido (Português): '{translated_new_sentence}'")
print("-" * 30)


new_english_sentence_2 = "He is a student."
new_input_seq_2 = input_tokenizer.texts_to_sequences([new_english_sentence_2])
new_input_data_2 = pad_sequences(new_input_seq_2, maxlen=max_encoder_seq_length, padding='post')

translated_new_sentence_2 = decode_sequence(new_input_data_2)
print(f"\nNova frase (Inglês): '{new_english_sentence_2}'")
print(f"Traduzido (Português): '{translated_new_sentence_2}'")
print("-" * 30)

print("\n--- Fim da Demonstração da Tradução ---")



--- Testando o Modelo de Tradução ---
Original (Inglês): 'I am a student.'
Esperado (Português): 'Eu sou um estudante.'
Traduzido (Português): 'eu sou um estudante.'
------------------------------
Original (Inglês): 'You are a teacher.'
Esperado (Português): 'Você é um professor.'
Traduzido (Português): 'você é um professor.'
------------------------------
Original (Inglês): 'He is a doctor.'
Esperado (Português): 'Ele é um médico.'
Traduzido (Português): 'ele é um médico.'
------------------------------
Original (Inglês): 'She is happy.'
Esperado (Português): 'Ela está feliz.'
Traduzido (Português): 'ela está feliz.'
------------------------------
Original (Inglês): 'We are friends.'
Esperado (Português): 'Nós somos amigos.'
Traduzido (Português): 'nós somos amigos.'
------------------------------
Original (Inglês): 'They are playing.'
Esperado (Português): 'Eles estão jogando.'
Traduzido (Português): 'eles estão jogando.'
------------------------------
Original (Inglês): 'The cat is